# Using LaVague for QA Automation

In this notebook, we'll show how LaVague can be used to automatically generate pytest files from a Gherkin test definition

We will use LaVague to autonomously run the test and record xpath and actions. We'll then use an LLM to generate assert statements and the final reusable test file. 

# Test cases
**There are currently two examples test cases, only run the cell you want to test**

## Scope packaging CLI


copier coller de lavague tests avec 

/lavague-qa/
    examples/
        demo_amazon.yaml
            URL
            GHERKIN
    src/

```
lavague-qa -url https://example.com -feature path/to/feature/file

```


## Test case 1: Amazon - ecommerce cart feature

LaVague can help you add test coverage for critical parts of your websites such as a Cart feature for ecommerce. 

- Accept cookies
- Perform a search
- Click on a product
- Add it to cart
- Access the cart page
- Remove the item from cart
- Verify that the cart is empty

In [ ]:
URL = "https://www.amazon.fr/"

GHERKIN = """Feature: Cart

  As a user, 
  I want, 
  So that, 

  Scenario: Add and remove a single product from cart
    Given I am on the homepage
    When I click "Accepter" to accept cookies
    And I enter "Zero to One" into the search bar and press Enter
    And I click on the first product in the search results
    And I click on the "Ajouter au panier" button
    And I the confirmation message has been displayed
    And I click on "Aller au panier" under "Passer la commande"
    And I click on "Supprimer" from the cart page
    Then the cart should be empty
"""

FEATURE_FILE_NAME = "demo_amazon.feature"
TEST_FILE_NAME = "demo_amazon.py"

## Test case 2: La Poste - interactive components

LaVague can test the proper behavior of interactive elements on your site

- Accept cookies
- Navigate to the shipping calculator
- Select values for package size and weight
- Verify the price is as expected

In [ ]:
URL = "https://www.laposte.fr/"

GHERKIN = """Feature: Shipping cost calculator

  Scenario: Estimate shipping costs for a large package
    Given I am on the homepage
    When I click on "J'accepte" to accept cookies
    And I click on "Envoyer un colis"
    And I click on the "Format du colis" dropdown under "Dimension"
    And I click on "Volumineux & tube" from the dropdown results
    And I enter 15 in the "Poids" field
    And I wait for the cost to update
    Then the cost should be "34,70 €"
"""

FEATURE_FILE_NAME = "demo_laposte.feature"
TEST_FILE_NAME = "demo_laposte.py"

## Test case 3: Wikipedia login

LaVague can test login features with some provided credentials

- Accept cookies
- Go to login
- Login using credentials
- Verifiy that login was successful

In [ ]:
URL = "https://fr.wikipedia.org/"

GHERKIN = """Feature: Wikipedia Login

  Scenario: User logs in successfully
    Given the user is on the Wikipedia homepage
    When the user navigates to the login page
    And the user enters Lavague-test in the username field
    And the user enters lavaguetest123 in the password field
    And the user submits the login form
    Then the login should be successful and the user is redirected to the main page
"""

FEATURE_FILE_NAME = "demo_wikipedia.feature"
TEST_FILE_NAME = "demo_wikipedia.py"

## Test case 4: HSBC Navigation

LaVague can make sure that links on your sites are not broken by checking multiple tabs if required

- Go on homepage
- Accept cookies
- Click on a redirection link
- Accept redirection
- Switch to tab 
- Accept cookies
- Navigate to another page
- Check if expected page

In [ ]:
URL = "https://www.hsbc.fr/"

GHERKIN = """Feature: HSBC navigation

  Scenario: Multi tab navigation
    Given the user is on the HSBC homepage
    When the user clicks on "Tout accepter" to accept cookies
    And the user clicks on "Global Banking and Markets"
    And the user clicks on "Je comprends, continuons"
    And the user navigates to the new tab opened
    And the user clicks on "Accept all cookies"
    And the user clicks on "About us"
    Then the user should be on the "About us" page of the "Global Banking and Markets" services of HSBC
"""

FEATURE_FILE_NAME = "demo_hsbc.feature"
TEST_FILE_NAME = "demo_hsbc.py"

## POC: End to end Gherkin generation

We'll give LaVague the task of crawling the site and coming up with test cases in the Gherkin format that we will generate code for. This way we can add coverage from high level instructions. 

In [ ]:
URL = "https://www.axa.fr/"
# OBJECTIVE = "Output one valid Gherkin file containing three scenarios for one feature to be tested on this site. Test steps should be precise and site specific. Keep the tests simple and make sure "
OBJECTIVE = "Output one valid Gherkin file containing three scenarios for one feature to be tested on this site. The feature you'll be outputting tests for should require no real world data and should be testable with synthetic data only. The three assert statements should be testable from the same end state for all scenarios. You can navigate to subpages to make sure you output precise site specific test steps."

FEATURE_FILE_NAME = "demo_endtoend.feature"
TEST_FILE_NAME = "demo_endtoend.py"

In [ ]:
from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver
from lavague.core.retrievers import SemanticRetriever
from selenium.webdriver.chrome.options import Options

from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.legacy.readers.file.base import SimpleDirectoryReader

import time

selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

agent.get(URL)
time.sleep(3)

agent_output = agent.run(OBJECTIVE, log_to_db=True)

logs = agent.logger.return_pandas()

In [ ]:
print(agent_output)

In [ ]:

def clean_output(markdown_code_block):
    return markdown_code_block.replace("```gherkin", "").replace("```", "").replace("```\n", "").replace("gherkin", "")

GHERKIN = clean_output(agent_output.output)

GHERKIN

In [ ]:
gpt3_5 = OpenAI("gpt-3.5-turbo")
output = gpt3_5.complete("Output a valid Gherkin file surrounded by triple quotes from this: " + GHERKIN, max_tokens=500).text


In [ ]:

def clean_output(markdown_code_block):
    return markdown_code_block.replace("```gherkin", "").replace("```", "").replace("```\n", "")

print(clean_output(output))
GHERKIN = clean_output(output)


In [ ]:
URL = "https://temp-mail.org/en/"

GHERKIN = """Get a temporary email address from this website then go on https://wwws.airfrance.fr/ and attempt to create an account. Go back to the temp-mail website and verify the email address if needed. 
"""

FEATURE_FILE_NAME = "demo_airfrance.feature"
TEST_FILE_NAME = "demo_airfrance.py"

# Running a test case with LaVague

## Parsing the Gherkin
We parse the Gherkin file to extract the assert statement, this will be usefull when we want to generate the assert code

In [ ]:
# scenarios = GHERKIN.split("Scenario:")
# parsed_scenarios = []
# for scenario in scenarios[1:]:
#     scenario_name, *scenario_steps = scenario.strip().split("\n")
#     parsed_scenarios.append(
#         {
#             "name": scenario_name.strip(),
#             "steps": [step.strip() for step in scenario_steps],
#         }
#     )

# def prepare_scenario(scenario):
#     return scenario["steps"], scenario["steps"][-1]


# steps, assert_statement = prepare_scenario(parsed_scenarios[0])

# print(assert_statement)
# print(steps)

In [ ]:
scenarios = GHERKIN.split("Scenario:")
parsed_scenarios = []
for scenario in scenarios[1:]:
    scenario_name, *scenario_steps = scenario.strip().split("\n")
    parsed_scenarios.append(
        {
            "name": scenario_name.strip(),
            "steps": [step.strip() for step in scenario_steps],
        }
    )

def prepare_scenario(scenario):
    return scenario["steps"], scenario["steps"][-1]

assert_statements =  [scenario['steps'][-1] for scenario in parsed_scenarios]
print(assert_statements)
print(parsed_scenarios)

## Init LaVague

We create a standard LaVague agent and open the URL

In [ ]:
from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver
from lavague.core.retrievers import SemanticRetriever
from selenium.webdriver.chrome.options import Options

from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.legacy.readers.file.base import SimpleDirectoryReader

selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

agent.get(URL)

## Start the agent
We start the agent and record the steps taken as a `pandas` dataframe

In [ ]:
objective = "Run these scenarios step by step, make sure you complete each step: " + GHERKIN
# objective = GHERKIN
# # agent.get("https://wwws.airfrance.fr/")
agent.run(objective, log_to_db=True)

logs = agent.logger.return_pandas()

# Processing a test case

## Extract run steps and code
After the agent has finished running, we extract instructions and actions taken from the logs. 
We also get the screenshot of the last page visited. 

In [ ]:
# we will remove chain of thought comments to only keep the actions
def remove_comments(code):
    return '\n'.join([line for line in code.split('\n') if not line.strip().startswith('#')])

# get all actions
actions = "\n".join(logs["code"].dropna())

# clean data
logs['action'] = logs['code'].dropna().apply(remove_comments)
cleaned_logs = logs[['instruction', 'action']].fillna('')
actions = '\n\n'.join(cleaned_logs['instruction'] + ' ' + cleaned_logs['action'])

# get last page screenshot
last_page_screenshot = SimpleDirectoryReader(logs.iloc[-1]["screenshots_path"]).load_data() # load last screenshot taken

print(actions)

## Retrieve nodes for assert
Using the `SemanticRetriever`, we fetch nodes that could be relevant to generate the assert statement

In [ ]:

# tester xpathed_only à false
retriever = SemanticRetriever(embedding=action_engine.python_engine.embedding, xpathed_only=False)
html = selenium_driver.get_html()
nodes = retriever.retrieve(f"{assert_statements}" , html.splitlines())

# evolution potentielle pour + de robustesse dans la génération d'assert: 
# - identifier le Xpath de l'element sur lequel faire l'assert (ajouter un autre retriever avant le SemanticRetriever)
# - modifier la pipeline retrieval pour tagger chaque element avec un xpath
# - passer ca dans un LLM + le assert pour identifier le xpath de l'element sur lequel faire l'assert. 


# tag all nodes with xpath -> semantic retrieve on the assert statement -> extract xpath from the node that matches the assert statement

In [ ]:
print(nodes)

In [ ]:
# you can run this cell to display all nodes returned by the retriever
from IPython.display import HTML

for e in nodes: 
    display(HTML(e))

# Generate the `pytest` file
Use recorded data about the site to generate a pytest-bdd file

We use a multi modal LLM (`gpt-4o`) to generate the final code file

In [ ]:
gpt4o = OpenAIMultiModal("gpt-4o")
gpt4o.max_new_tokens = 2000 # 300 by default, we increase it to make sure our pytest file doesn't get trucated

# we'll clean the triple quoted answers from the LLM
def clean_output(markdown_code_block):
    return markdown_code_block.replace("```python", "").replace("```", "").replace("```\n", "")

## Building the prompt

We use a prompt that combines general instructions, examples and the recorded run data to generate the pytest file

In [ ]:
EXAMPLE_PYTEST = """import pytest
from pytest_bdd import scenarios, given, when, then, parsers
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time
import random

# Constants
BASE_URL = 'https://example.com'

# Scenarios
scenarios('complex_example.feature')

# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    yield driver
    driver.quit()

# Steps
@given('I am on the example website')
def go_to_homepage(browser):
    browser.get(BASE_URL)

@when('I navigate to the product catalog')
def navigate_to_catalog(browser):
    catalog_link = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/header/nav/ul/li[3]/a"))
    )
    try:
        browser.execute_script("arguments[0].click();", catalog_link)
    except ElementClickInterceptedException:
        pytest.fail("Failed to navigate to the product catalog")

@when('I filter products by category')
def filter_products(browser):
    category_dropdown = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/main/div/div[1]/aside/div[3]/select"))
    )
    options = category_dropdown.find_elements(By.TAG_NAME, "option")
    random_option = random.choice(options[1:])  # Exclude the first option if it's a placeholder
    random_option.click()

@when('I sort products by price')
def sort_products(browser):
    sort_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/main/div/div[2]/div[1]/div/button[2]"))
    )
    try:
        browser.execute_script("arguments[0].click();", sort_button)
    except ElementClickInterceptedException:
        pytest.fail("Failed to sort products")

@when('I add a random product to the cart')
def add_to_cart(browser):
    products = browser.find_elements(By.XPATH, "/html/body/div[2]/main/div/div[2]/ul/li")
    random_product = random.choice(products)
    add_to_cart_button = random_product.find_element(By.XPATH, ".//button[@data-testid='add-to-cart']")
    try:
        browser.execute_script("arguments[0].click();", add_to_cart_button)
    except ElementClickInterceptedException:
        pytest.fail("Failed to add product to cart")

@when('I proceed to checkout')
def proceed_to_checkout(browser):
    checkout_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/header/div[2]/div/div/a"))
    )
    try:
        browser.execute_script("arguments[0].click();", checkout_button)
    except ElementClickInterceptedException:
        pytest.fail("Failed to proceed to checkout")

@then('I should see the checkout form')
def verify_checkout_form(browser):
    try:
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.ID, "checkout-form"))
        )
    except Exception as e:
        pytest.fail(f"Checkout form not found: {str(e)}")

@then('the cart total should be correct')
def verify_cart_total(browser):
    try:
        total_element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.ID, "cart-total"))
        )
        total_value = float(total_element.text.replace('$', ''))
        assert total_value > 0, "Cart total should be greater than zero"
    except Exception as e:
        pytest.fail(f"Failed to verify cart total: {str(e)}")

@then('the product list should be visible')
def verify_product_list(browser):
    try:
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-list"))
        )
    except Exception as e:
        pytest.fail(f"Product list not found: {str(e)}")

@then('the category filter should be available')
def verify_category_filter(browser):
    try:
        filter_element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.ID, "category-filter"))
        )
        assert filter_element.is_enabled(), "Category filter should be enabled"
    except Exception as e:
        pytest.fail(f"Category filter not found or not enabled: {str(e)}")

@then('the "Add to Cart" button should be present for each product')
def verify_add_to_cart_buttons(browser):
    try:
        add_to_cart_buttons = WebDriverWait(browser, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button[data-testid='add-to-cart']"))
        )
        assert len(add_to_cart_buttons) > 0, "No 'Add to Cart' buttons found"
    except Exception as e:
        pytest.fail(f"Failed to verify 'Add to Cart' buttons: {str(e)}")
"""


PROMPT = f"""
You are an expert in software testing frameworks and Python code generation. You answer in python markdown only and nothing else.
Your only goal is to generate pytest-bdd files based on the provided Gherkin feature, a collection of instructions and actions, and a specific assert statement to test.
You will use the provided information to generate a valid assert statement. 
- Name the scenario appropriately.
- Always use time.sleep(3) if waiting is required.
- Include all necessary imports and fixtures.
- Use provided actions to find valid XPath selectors for the final pytest file. 
- You answer in python code only and nothing else.

I will provide an example below:
----------
Feature file name: example.feature

URL: https://www.example.com

Gherkin of the feature to be tested: 

Feature: E-commerce Website Interaction

  Scenario: Browse products and checkout
    Given I am on the example website
    When I navigate to the product catalog
    And I filter products by category
    And I sort products by price
    And I add a random product to the cart
    And I proceed to checkout
    Then the cart total should be correct

Assert statement Then the cart total should be correct

List of already executed instructions and actions:
- instruction: Navigate to the product catalog page
  actions:
    - action:
        name: "click"
        args:
          xpath: "/html/body/div[1]/header/nav/ul/li[3]/a"
          value: ""

- instruction: Select a category from the dropdown filter
  actions:
    - action:
        name: "click"
        args:
          xpath: "/html/body/div[2]/main/div/div[1]/aside/div[3]/select/option[3]"
          value: ""

- instruction: Sort products by price
  actions:
    - action:
        name: "click"
        args:
          xpath: "/html/body/div[2]/main/div/div[2]/div[1]/div/button[2]"
          value: ""

- instruction: Add a random product to the cart
  actions:
    - action:
        name: "click"
        args:
          xpath: "/html/body/div[2]/main/div/div[2]/ul/li[5]/div/button[@data-testid='add-to-cart']"
          value: ""

- instruction: Proceed to checkout
  actions:
    - action:
        name: "click"
        args:
          xpath: "/html/body/div[1]/header/div[2]/div/div/a"
          value: ""
          
          
Resulting pytest code: 
{EXAMPLE_PYTEST}

----------
Given this information, generate a valid pytest-bdd file with the following inputs:
Feature file name: {FEATURE_FILE_NAME}\n
URL: {URL}\n
Gherkin of the feature to be tested:
{GHERKIN}\n
Assert statement: {assert_statements}\n
Potentially relevant nodes that you may use to help you generate the assert code: {nodes}\n
List of already executed instructions and actions:
{actions}\n
"""

In [ ]:
print(PROMPT)

## Generate the test
Call the LLM with the final prompt then clean the ouput

In [ ]:
generated_pytest = gpt4o.complete(PROMPT, image_documents=last_page_screenshot).text
generated_pytest = clean_output(generated_pytest)

In [ ]:
print(generated_pytest)

## Write test to file

We'll write two generated files to disk: 
- `.feature` contains the test scenarios written in the Gherkin syntax
- `.py` the actual automated test that we'll run with pytest

In [ ]:
import os
with open(FEATURE_FILE_NAME, "w") as file:
        file.write(GHERKIN)
        
with open(TEST_FILE_NAME, "w") as file:
        print("WRITING FILE")
        file.write(generated_pytest)

# Run tests

We can finally run our generated test to see it in action

In [ ]:
!pytest demo_amazon.py

In [ ]:
!pytest demo_laposte.py

In [ ]:
!pytest demo_wikipedia.py

In [ ]:
!pytest demo_hsbc.py

In [ ]:
!pytest -v demo_endtoend.py